In [2]:
import numpy as np
import matplotlib.pyplot as plt

def loadDataSet():#按格式读取数据
    dataMat = []
    labelMat = []
    fr = open('testSet.txt')
    for line in fr.readlines():#按行读取
        lineArr = line.strip().split()#分割数据，# 分割数据,split默认参数，会自己识别分隔符
        dataMat.append([1.0, float(lineArr[0]), float(lineArr[1])])#样本数据
        labelMat.append(int(lineArr[2]))#标签数据
    return dataMat, labelMat


def sigmoid(inX):#sigmoid函数
    if inX >= 0:
        return 1.0 / (1 + np.exp(-inX))
    else:
        return np.exp(inX) / (1 + np.exp(inX))#sigmoid函数有坑，数据过小会报错，所以需要调整一下函数形式


def draw_sigmoid():#测试用的，画出sigmoid的图形
    x = np.arange(-10,10)
    y = sigmoid(x)
    plt.plot(x,y)
    plt.show()



def gradAscent(dataMatIn, classLabels):#梯度上升算法
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()
    m, n = np.shape(dataMatrix)
    alpha = 0.01
    maxCycles = 500
    weights = np.ones([n, 1])#参数一开始都置为1，这个方法似乎不太好，应该重新为设置为随机的很小的数
    for k in np.arange(maxCycles):#设置学习次数
        h = sigmoid(dataMatrix * weights)
        error = (labelMat - h)#计算误差
        weights = weights + alpha * dataMatrix.transpose() * error#梯度上升weights的更新方法
    return weights


def plotBestFit(weights):#画出最优分类曲线
    dataMat, labelMat = loadDataSet()
    dataArr = np.array(dataMat)
    n = np.shape(dataArr)[0]
    xcord1 = []
    xcord2 = []
    ycord1 = []
    ycord2 = []
    for i in np.arange(n):
        if int(labelMat[i]) == 1:
            xcord1.append(dataArr[i,1]);ycord1.append(dataArr[i,2])
        else:
            xcord2.append(dataArr[i, 1]);ycord2.append(dataArr[i,2])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(xcord1, ycord1, s = 30 ,c = 'red', marker = 's')
    ax.scatter(xcord2, ycord2, s = 30, c = 'green')
    x = np.arange(-3.0, 3.0, 0.1)
    y = (-weights[0] - weights[1]*x)/weights[2]
    #y = np.array(y)[0]
    ax.plot(x,y)
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.show()


# def stocGradAscent1(dataMatIn, classLabels, numIter=150):
#     dataMatrix = np.array(dataMatIn)
#     m, n = np.shape(dataMatIn)
#     weights = np.ones(n)   #weights为numpy.ndarray类型
#     for j in range(numIter):
#         dataIndex = list(range(m))
#         for i in range(m):
#             alpha = 4/(1.0+j+i)+0.0001    #apha decreases with iteration, does not go to 0 because of the constant
#             randIndex = int(np.random.uniform(0,len(dataIndex)))
#             theChosenOne = dataIndex[randIndex]
#             h = sigmoid(sum(dataMatrix[theChosenOne]*weights))
#             error = classLabels[theChosenOne] - h
#             weights = weights + alpha * error * dataMatrix[theChosenOne]
#             del(dataIndex[randIndex])
#     return weights

def stocGradAscent1(dataMatrix, classLabels, numIter = 150):#随机梯度上升方法
    dataMatrix = np.array(dataMatrix)
    m, n = np.shape(dataMatrix)
    weights = np.ones(n)
    for j in np.arange(numIter):
        randIndex = np.arange(m)
        np.random.shuffle(randIndex)
        for i in range(m):
            alpha = 4/(1.0 + j + i) + 0.01#这里有个疑问，0.01似乎有些大了
            h = sigmoid(sum(dataMatrix[randIndex[i]]*weights))
            error = classLabels[randIndex[i]] - h
            weights = weights + alpha * error * dataMatrix[randIndex[i]]#随机梯度上升方法的weights更新方法
    return weights


def classifyVector(inX, weights):#测试函数，0.5作为sigmoid函数的分类标准
    prob = sigmoid(np.sum(inX*weights))
    if prob > 0.5 : return 1.0
    else : return 0.0


def colicTest():#疝气病症预测病马的死亡率
    frTrain = open('horseColicTraining.txt')
    frTest = open('horseColicTest.txt')
    trainingSet = []
    trainingLabels = []
    for line in frTrain.readlines():
        currLine = line.strip().split('\t')
        lineArr = []
        for i in range(21):
            lineArr.append(float(currLine[i]))
        trainingSet.append(lineArr)
        trainingLabels.append(float(currLine[21]))
    trainWeights = stocGradAscent1(np.array(trainingSet), trainingLabels, 500)
    errorCount = 0
    numTestVec = 0.0
    for line in frTest.readlines():
        numTestVec += 1.0
        currLine = line.strip().split('\t')
        lineArr = []
        for i in range(21):
            lineArr.append(float(currLine[i]))
        if int(classifyVector(np.array(lineArr), trainWeights)) != int(currLine[21]):#记录预测与实际不符的样本数
            errorCount += 1
    errorRate =(float(errorCount)/numTestVec)
    print("the error rate of this test is :{}".format(errorRate))
    return errorRate


def multiTest():
    numTests = 10
    errorSum = 0.0
    for k in range(numTests):
        errorSum += colicTest()
    print("after {} iterations the average error rate is: {}".format(numTests, errorSum/float(numTests)))


multiTest()
# dataArr ,labelMat = loadDataSet()
# # weights = gradAscent(dataArr, labelMat)
# # weights = stocGradAscent1(dataArr, labelMat)
# # print(weights)
# # plotBestFit(weights)

the error rate of this test is :0.23880597014925373
the error rate of this test is :0.2835820895522388
the error rate of this test is :0.3582089552238806
the error rate of this test is :0.3880597014925373
the error rate of this test is :0.3582089552238806
the error rate of this test is :0.23880597014925373
the error rate of this test is :0.31343283582089554
the error rate of this test is :0.31343283582089554
the error rate of this test is :0.2835820895522388
the error rate of this test is :0.43283582089552236
after 10 iterations the average error rate is: 0.32089552238805974
